In [1]:
# import ssl
# import urllib.request
# import torch
# from PIL import Image
# import matplotlib.pyplot as plt
# from torchvision import transforms
# from torch.nn import functional as F
# %load_ext autoreload

# %autoreload 2

# ssl._create_default_https_context = ssl._create_unverified_context
# from src.anti_spoof.fas import flip_it

from deepface import DeepFace
import re

import streamlit as st

In [ ]:
# model = flip_it()
# model_path = "pretrained_models/casia_flip_mcl.pth.tar"
# checkpoint = torch.load(model_path, map_location="cpu")
# model.load_state_dict(checkpoint["state_dict"], strict=False)
# model.eval()

In [ ]:
# img1_path = "../spoof/spoof/spoof_314.png"
# img = Image.open(img1_path)
# plt.imshow(img)

In [3]:
# preprocess = transforms.Compose(
# [
# transforms.Resize([224, 224]),
# transforms.ToTensor(),
# transforms.Normalize(mean=[0.485], std=[0.229])
# ]
# )

In [2]:
def authorise_user(image_path):
    print('\n-------------- Authorisation --------------\n')

    # anti spoof
    # img = Image.open(image_path)

    # input = preprocess(img).unsqueeze(0)
    # cls_out, feature = model(input, norm_flag=True)
    # prob = F.softmax(cls_out, dim=1).cpu().data.numpy()

    # spoof_result = ((prob[:, 1])>= 0.5).astype(int)[0]

    face_objs = DeepFace.extract_faces(
    img_path=image_path,
    anti_spoofing = True
    )

    if not all(face_obj["is_real"] is True for face_obj in face_objs):
        return 'Authorisation DENIED, spoof detected'

    else:
        result = DeepFace.find(image_path, db_path='./database')

        distance = result[0]['distance'][0]
        person = ' '.join((result[0]['identity'][0]).split('\\')[-1].split('_')[:2])
        max_distance = 0.5

        print(f'Distance: {distance}')
        if distance <= max_distance:
            print(f'Authorisation confirmed. Welcome {person}!')
        else:
            print('Authorisation DENIED')
            
        return result

In [3]:
result = authorise_user('../dataset/Adam_Sandler/test_positive/Adam_Sandler_1088_560.jpeg')



-------------- Authorisation --------------

24-06-16 10:24:57 - Searching ../dataset/Adam_Sandler/test_positive/Adam_Sandler_1088_560.jpeg in 5323 length datastore
24-06-16 10:25:04 - find function duration 13.509296417236328 seconds
Distance: 0.09274071399858375
Authorisation confirmed. Welcome Adam Sandler!


In [4]:
from deepface import DeepFace
face_objs = DeepFace.extract_faces(
  img_path="../spoof/real/real_0.png",
  anti_spoofing = True
)

In [2]:
# assert all(face_obj["is_real"] is True for face_obj in face_objs)

AssertionError: 

In [5]:
# for face_obj in face_objs:
#     print(face_obj["is_real"])

if not all(face_obj["is_real"] is True for face_obj in face_objs):
    print('spoof')
else:
    print('no spoof')


no spoof
